In [ ]:
import random

import numpy as np
from numpy.linalg import norm
from sklearn.decomposition import PCA

from rdkit import Chem
from rdkit.Chem import rdDistGeom, rdForceFieldHelpers
from rdkit.Geometry import Point3D
import py3Dmol

def find_min_conformer(smiles, num_conf: int = 100, max_opt_iters: int = 1000):
    print("\n----------------------------------------------------------------")
    print("Generating initial CREST input structure:\n")

    molecule = Chem.MolFromSmiles(smiles)

    molecule_h = Chem.AddHs(molecule)
    Chem.rdCoordGen.AddCoords(molecule_h)

    print(
        f"Generating {num_conf} conformers for initial sorting and "
        "optimization using RDKit."
    )
    rdDistGeom.EmbedMultipleConfs(
        molecule_h,
        num_conf,
        params=rdDistGeom.ETKDGv3()
    )
    conf_energy = rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(
        molecule_h,
        maxIters=max_opt_iters,
        ignoreInterfragInteractions=False
    )

    if all(conf_set[0] == 0 for conf_set in conf_energy):
        print("All conformers converged.")
    else:
        print("WARNING! Not all conformers converged.")

    min_energy = float("inf")
    min_index = 0
    for index, (_, energy) in enumerate(conf_energy):
        if energy < min_energy:
            min_energy = energy
            min_index = index
    mol_min = Chem.Mol(molecule_h, False, min_index)

    return mol_min

def orientate_rod(molecule):
    conf = molecule.GetConformer()
    xyz_pos = conf.GetPositions()
    mean = xyz_pos.mean(axis=0)
    centered = xyz_pos - mean

    pca = PCA(n_components=3)
    pca.fit(centered)
    rod_axis = pca.components_[0]
    z_axis = np.array([0, 0, 1])

    a, b = (rod_axis / norm(rod_axis)).reshape(3), (z_axis / norm(z_axis)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    
    for i in range(molecule.GetNumAtoms()):
        pos = conf.GetAtomPosition(i)
        vec = np.array([pos.x, pos.y, pos.z])
        rotated_vec = rotation_matrix @ vec
        conf.SetAtomPosition(i, rotated_vec)

    return molecule

def translate_mol(molecule, dx=0, dy=0, dz=0):

    molecule = Chem.Mol(molecule)
    conf = molecule.GetConformer()
    for i in range(molecule.GetNumAtoms()):
        pos = conf.GetAtomPosition(i)
        conf.SetAtomPosition(i, Point3D(pos.x + dx, pos.y + dy, pos.z + dz))
    return molecule

def rotate_mol(molecule, angle_deg):

    molecule = Chem.Mol(molecule)
    conf = molecule.GetConformer()
    angle_rad = np.radians(angle_deg)
    cos_angle = np.cos(angle_rad)
    sin_angle = np.sin(angle_rad)
    
    for i in range(molecule.GetNumAtoms()):
        pos = conf.GetAtomPosition(i)
        x_new = cos_angle * pos.x - sin_angle * pos.y
        y_new = sin_angle * pos.x + cos_angle * pos.y
        conf.SetAtomPosition(i, Point3D(x_new, y_new, pos.z))
    return molecule

def tilt_mol(molecule, tilt_angle):
    conf = molecule.GetConformer()
    xyz_pos = conf.GetPositions()
    mean = xyz_pos.mean(axis=0)

    # 1. Choose random tilt angle and azimuth
    tilt_angle_rad = np.radians(tilt_angle)
    azimuth = np.radians(np.random.uniform(0, 360))

    # 2. Define axis in x–y plane to tilt around
    tilt_axis = np.array([np.cos(azimuth), np.sin(azimuth), 0.0])

    # 3. Rodrigues' rotation matrix
    v = tilt_axis / norm(tilt_axis)
    s = np.sin(tilt_angle_rad)
    c = np.cos(tilt_angle_rad)
    kmat = np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])
    rotation_matrix = np.eye(3) + s * kmat + (1 - c) * kmat @ kmat

    # 4. Apply rotation to each atom
    for i in range(molecule.GetNumAtoms()):
        pos = np.array(conf.GetAtomPosition(i)) - mean
        rotated = rotation_matrix @ pos + mean
        conf.SetAtomPosition(i, rotated)

    return molecule

def build_cluster(mol, n_mols, radius=6.0, min_dist=3, max_attempts=1000):

    mols = [mol]  # Central molecule
    existing_coords = list(mol.GetConformer().GetPositions())

    for i in range(n_mols):
        for attempt in range(max_attempts):
            angle_deg = i * 360/n_mols
            angle_rad = np.radians(angle_deg)
            dx = radius * np.cos(angle_rad) + random.uniform(-2, 2)
            dy = radius * np.sin(angle_rad) + random.uniform(-2, 2)
            dz = random.uniform(-3, 3)
            tilt = random.uniform(0, 10)
            
            dtheta = random.uniform(0, 360)
            rotated = rotate_mol(mol, dtheta)
            tilted = tilt_mol(rotated, tilt)
            translated = translate_mol(tilted, dx=dx, dy=dy, dz=dz)

            new_coords = list(translated.GetConformer().GetPositions())

            # Check for overlaps
            too_close = any(
                np.linalg.norm(pos1 - pos2) < min_dist
                for pos1 in new_coords
                for pos2 in existing_coords
            )

            if not too_close:
                mols.append(translated)
                existing_coords.extend(new_coords)
                break
        else:
            print(f"Warning: Could not place molecule {i+1} without overlap.")
            return None

    # Combine all molecules into one
    combined = mols[0]
    for m in mols[1:]:
        combined = Chem.CombineMols(combined, m)
    
    return combined

def show_mol_3d(mol, style='stick'):
    mb = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=400, height=400)
    viewer.addModel(mb, 'mol')
    viewer.setStyle({style: {}})
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    return viewer.show()

smiles = "O=C(OC1=CC(F)=C(C2=CC=C(C#N)C(F)=C2)C(F)=C1)C(C(F)=C3)=CC=C3CCC"
mol = find_min_conformer(smiles, num_conf=10)
mol = orientate_rod(mol)
mol_cluster = build_cluster(mol, 6, radius=5.0)
while mol_cluster == None:
    mol_cluster = build_cluster(mol, 6, radius=5.0)
show_mol_3d(mol_cluster)


----------------------------------------------------------------
Generating initial CREST input structure:

Generating 10 conformers for initial sorting and optimization using RDKit.
All conformers converged.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.